In [ ]:
#####################################################
############EJERCICIO1###############################
#PAQUETES PARA QUE FUNCIONE EL PROGRAMA

#   1era  parte comparación genómica

# BioPython: Para parsear resultados BLAST (NCBIXML)
# pip install biopython

# CSV: Para generar archivos CSV (incluido en Python estándar, no necesita instalación)
# import csv

# OS y subprocess: Para manejo de archivos y ejecutar comandos (incluidos en Python estándar)
# import os
# import subprocess

#  2da parte

# Para generación de documentos Word (reportes):
# pip install python-docx

# ===== HERRAMIENTAS EXTERNAS REQUERIDAS =====
# BLAST+ (debe instalarse manualmente)


In [ ]:
# === UMBRALES DE FILTRADO PARA BLAST ===
#IMPORTANTE!!!!!!!!!


#LEEEER!!!1

#LOS DATOS SIGUIENTES SON AJUSTABLES!!!!.POR EJEMPLO SI SE TRABAJA CON IDENTIDAD AL 100% Y UNA COBERTURA ==1.0
#LOS RESULTADOS MOSTRARÁN QUE (((NINGÚN)))) GENOMA TIENE EL GEN RESISTENTE

# Identidad > 95%:
# - Este valor garantiza que la secuencia alineada es casi idéntica al gen de referencia (blatem1/blactxm15).
# - En resistencia bacteriana, mutaciones puntuales menores (<5%) no suelen afectar la función del gen.
# - Evita falsos positivos por secuencias similares pero no relacionadas (ej: genes homólogos no resistentes).

# Cobertura > 0.9 (90%):
# - Asegura que al menos el 90% del gen de referencia esté presente en el genoma analizado.
# - Descarta fragmentos cortos que podrían alinearse por casualidad pero no representan el gen completo.
# - Especialmente importante para genes grandes donde regiones parciales podrían ser no funcionales.

# Combinación de ambos criterios:
# - Identidad alta + cobertura alta = alta confianza en que el gen resistente está presente y funcional.

In [ ]:
####=====GUIA====###
#EJECUTAR EN ORDEN
#1ero el 1er bloque de código "COMPARACIÓN GENÓMICA"
#2do el 2do bloque de código "GENERACIÓN DE INFORMES"

In [41]:
######COMPARACIÓN GENÓMICA######
import os
import subprocess
from Bio.Blast import NCBIXML
import csv
# Añadir al inicio del script
from datetime import datetime

def generar_reporte_html(resultados):
    """Genera un reporte visual interactivo con los resultados"""
    fecha = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Reporte de Resistencia Antibiótica</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            h1 {{ color: #2c3e50; }}
            .positivo {{ color: #e74c3c; font-weight: bold; }}
            .negativo {{ color: #2ecc71; }}
            table {{ border-collapse: collapse; width: 100%; }}
            th, td {{ padding: 8px; text-align: left; border-bottom: 1px solid #ddd; }}
            tr:hover {{ background-color: #f5f5f5; }}
        </style>
    </head>
    <body>
        <h1>Reporte de Análisis - {fecha}</h1>
        <table>
            <tr>
                <th>Genoma</th>
                <th>blaTEM-1</th>
                <th>blaCTX-M-15</th>
            </tr>
    """

    for r in resultados:
        html += f"""
            <tr>
                <td>{r['archivo']}</td>
                <td class="{'positivo' if r['blatem1'] else 'negativo'}">
                    {'✅ Presente' if r['blatem1'] else '❌ Ausente'}
                </td>
                <td class="{'positivo' if r['blactxm15'] else 'negativo'}">
                    {'✅ Presente' if r['blactxm15'] else '❌ Ausente'}
                </td>
            </tr>
        """

    html += """
        </table>
    </body>
    </html>
    """

    with open("reporte_resistencia.html", "w", encoding="utf-8") as f:
        f.write(html)
    print("[INFO] Reporte HTML generado: reporte_resistencia.html")




# === Configuración ===
GEN_BLATEM1 = "blatem1.fasta"
GEN_BLACTXM15 = "blactxm15.fasta"
TEMP_DIR = "./temp_blast_db"

os.makedirs(TEMP_DIR, exist_ok=True)

# Archivos que ignoramos al buscar genomas
ARCHIVOS_IGNORADOS = {GEN_BLATEM1, GEN_BLACTXM15}


def crear_blast_db(genoma_path, db_prefix):
    """Crea una base de datos BLAST local"""
    cmd = [
        "makeblastdb",
        "-in", genoma_path,
        "-dbtype", "nucl",
        "-out", db_prefix
    ]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


def ejecutar_blast(query_file, db_prefix):
    resultado_xml = os.path.join(TEMP_DIR, "resultado.xml")
    subprocess.run(["blastn", "-query", query_file, "-db", db_prefix, "-out", resultado_xml, "-outfmt", "5"])

    with open(resultado_xml) as result_file:
        blast_record = NCBIXML.read(result_file)

    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            identidad = (hsp.identities / hsp.align_length) * 100
            cobertura = hsp.align_length / blast_record.query_length
            
            # --- NUEVO: Guardar detalles en un archivo ---
            with open("detalles_alineamientos.txt", "a") as log:
                log.write(f"\n** Genoma: {db_prefix} **\n")
                log.write(f"Contig: {alignment.hit_id}\n")
                log.write(f"Posición: {hsp.sbjct_start}-{hsp.sbjct_end}\n")
                log.write(f"Identidad: {identidad:.1f}% | Cobertura: {cobertura:.1%}\n")
                log.write(f"Secuencia alineada: {hsp.sbjct[:100]}...\n")  # Primeros 100 caracteres
            
            if hsp.expect < 0.05 and identidad > 95 and cobertura > 0.9:
                return True
    return False


def limpiar_temporales(db_prefix):
    """Borra archivos temporales de BLAST"""
    for ext in [".nhr", ".nin", ".nsq"]:
        temp_file = db_prefix + ext
        if os.path.exists(temp_file):
            os.remove(temp_file)


def main():
    resultados = []
    db_prefix = os.path.join(TEMP_DIR, "temp_db")

    # Leer todos los archivos del directorio actual
    for archivo in os.listdir("."):
        if archivo.endswith(".fna") or archivo.endswith(".fasta"):
            if archivo in ARCHIVOS_IGNORADOS:
                continue  # Saltar los genes de resistencia

            ruta_genoma = os.path.join(".", archivo)
            print(f"\n[INFO] Analizando genoma: {archivo}")

            limpiar_temporales(db_prefix)
            crear_blast_db(ruta_genoma, db_prefix)

            with open("detalles_alineamientos.txt", "a") as log:
                log.write(f"\n=== GENOMA ANALIZADO: {archivo} ===\n")

            tiene_blatem1 = ejecutar_blast(GEN_BLATEM1, db_prefix)
            tiene_blactxm15 = ejecutar_blast(GEN_BLACTXM15, db_prefix)

            resultados.append({
                "archivo": archivo,
                "blatem1": tiene_blatem1,
                "blactxm15": tiene_blactxm15
            })

    # Mostrar resultados finales
    print("\n{:<50} {:<10} {:<10}".format("Genoma", "blatem1", "blactxm15"))
    print("-" * 75)
    for r in resultados:
        print("{:<50} {:<10} {:<10}".format(
            r["archivo"],
            "✅ Sí" if r["blatem1"] else "❌ No",
            "✅ Sí" if r["blactxm15"] else "❌ No"
        ))
    with open("resultados_resistencia.csv", "w", newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Genoma', 'blatem1', 'blactxm15']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for r in resultados:
            writer.writerow({
                'Genoma': r['archivo'],
                'blatem1': 'Sí' if r['blatem1'] else 'No',
                'blactxm15': 'Sí' if r['blactxm15'] else 'No'
            })
    print("[INFO] Resultados guardados en: resultados_resistencia.csv")
    
    generar_reporte_html(resultados)
if __name__ == "__main__":
    main()


[INFO] Analizando genoma: GCF_050389795.1_MS10495_ARL02201_ST410_genomic.fna

[INFO] Analizando genoma: GCF_050516345.1_ASM5051634v1_genomic.fna

[INFO] Analizando genoma: GCF_050516355.1_ASM5051635v1_genomic.fna

[INFO] Analizando genoma: GCF_050516365.1_ASM5051636v1_genomic.fna

[INFO] Analizando genoma: GCF_050516375.1_ASM5051637v1_genomic.fna

[INFO] Analizando genoma: GCF_050516385.1_ASM5051638v1_genomic.fna

[INFO] Analizando genoma: GCF_050563505.1_ASM5056350v1_genomic.fna

[INFO] Analizando genoma: GCF_050568845.1_2848_genomic.fna

Genoma                                             blatem1    blactxm15 
---------------------------------------------------------------------------
GCF_050389795.1_MS10495_ARL02201_ST410_genomic.fna ❌ No       ❌ No      
GCF_050516345.1_ASM5051634v1_genomic.fna           ✅ Sí       ❌ No      
GCF_050516355.1_ASM5051635v1_genomic.fna           ❌ No       ❌ No      
GCF_050516365.1_ASM5051636v1_genomic.fna           ✅ Sí       ❌ No      
GCF_05051637

In [42]:
####=====GENERACIÓN DE INFORME=======#####
from docx import Document
from docx.shared import Inches, Pt, Cm, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH, WD_PARAGRAPH_ALIGNMENT
from docx.oxml.shared import OxmlElement, qn
from docx.oxml.ns import nsdecls
import datetime
import csv
import os

def add_highlight(run, color_hex):
    """Función para agregar resaltado a un run de texto"""
    # Convertir hex a RGB
    if color_hex == 'FFFF00':  # Amarillo
        run.font.highlight_color = 7
    elif color_hex == '00FF00':  # Verde claro
        run.font.highlight_color = 4
    elif color_hex == 'FF0000':  # Rojo claro
        run.font.highlight_color = 6

def leer_resultados_csv(archivo_csv):
    """Lee el archivo CSV y retorna una lista de diccionarios con los datos"""
    resultados = []
    try:
        with open(archivo_csv, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Extraer el nombre de la cepa del nombre del archivo genómico
                nombre_completo = row['Genoma']
                nombre_cepa = nombre_completo.replace('_genomic.fna', '')
                # Buscar el patrón que contiene el nombre de la cepa
                #if '_' in nombre_completo:
                    #partes = nombre_completo.split('_')
                    #if len(partes) >= 2:
                        #nombre_cepa = partes[1]  # Tomar la segunda parte
                    #else:
                        #nombre_cepa = partes[0]
                #else:
                    #nombre_cepa = nombre_completo.replace('.fna', '')
                
                resultados.append({
                    'nombre_cepa': nombre_cepa,
                    'genoma_completo': nombre_completo,
                    'blatem1': row['blatem1'],
                    'blactxm15': row['blactxm15']
                })
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {archivo_csv}")
        return []
    except Exception as e:
        print(f"Error al leer el archivo CSV: {e}")
        return []
    
    return resultados

def crear_informe_correlacion(nombre_cepa, resultado_blatem1, resultado_blactxm15):
    # Crear documento
    doc = Document()
    # Configurar tamaño de hoja OFICIO (Legal) - 8.5" x 14"
    section = doc.sections[0]
    section.page_width = Inches(8.5)
    section.page_height = Inches(14)
    # Configurar márgenes
    sections = doc.sections
    for section in sections:
        section.top_margin = Inches(1)
        section.bottom_margin = Inches(1)
        section.left_margin = Inches(1.2)
        section.right_margin = Inches(1.2)
    
    # Título principal centrado y en mayúsculas
    titulo = doc.add_paragraph()
    titulo.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run_titulo = titulo.add_run("MODELO DE INFORME DE CORRELACIÓN\nCLÍNICA ADMINISTRATIVA DEL SUS")
    run_titulo.font.name = 'Tahoma'
    run_titulo.font.size = Pt(10)
    run_titulo.bold = True
    # Configuración de indentación (1.5 cm)
    indentacion = Cm(1.5)
    
    # Información en formato de texto (no tabla)
    # A:
    p_a = doc.add_paragraph()
    p_a.paragraph_format.left_indent = indentacion
    p_a.add_run("A:").bold = True
    p_a.add_run("\t\tDIRECTOR MUNICIPAL DE SALUD GAMLP\n\t\tGOBIERNO AUTÓNOMO MUNICIPAL DE LA PAZ").bold = True
    
    # VÍA:
    p_via = doc.add_paragraph()
    p_via.paragraph_format.left_indent = indentacion
    doc.add_paragraph()
    p_via.add_run("VÍA:").bold = True
    p_via.add_run("\t\tLABORATORIO FARMACOLÓGICO").bold = True
    
    # DE:
    p_de = doc.add_paragraph()
    p_de.paragraph_format.left_indent = indentacion
    p_de.add_run("DE:").bold = True
    run_de = p_de.add_run("\t\tRESPONSABLE DE LA ELABORACIÓN DEL INFORME\n")
    run_de.bold = True
    run_de2 = p_de.add_run("\t\tBIOINGENIERO VALERIA AVRIL CANDIA ZARCO")
    run_de2.bold = True
    doc.add_paragraph()
    
    # REF.:
    p_ref = doc.add_paragraph()
    p_ref.paragraph_format.left_indent = indentacion
    p_ref.add_run("REF.:").bold = True
    run_ref1 = p_ref.add_run("\t\tINFORME DE REVISIÓN DEL CUMPLIMIENTO DE INFORME\n")
    run_ref1.bold = True
    run_ref2 = p_ref.add_run("\t\tGENÉTICO DE LA BACTERIA E. COLI\n ")
    run_ref2.bold = True
    run_ref3 = p_ref.add_run(f"\t\t{nombre_cepa}\n")  # Aquí se coloca el nombre de la cepa
    run_ref3.bold = True
    #add_highlight(run_ref3, 'FFFF00')  # Yellow highlight
    run_ref4 = p_ref.add_run("\t\tREALIZADA EN EL HOSPITAL MUNICIPAL DE SEGUNDO\n")
    run_ref4.bold = True
    run_ref5 = p_ref.add_run("\t\tNIVEL LA PORTADA CORRESPONDIENTE AL MES DE JUNIO\n")
    run_ref5.bold = True
    run_ref6 = p_ref.add_run("\t\tDE LA GESTIÓN 2025")
    run_ref6.bold = True
    doc.add_paragraph()
    
    # Fecha:
    p_fecha = doc.add_paragraph()
    p_fecha.paragraph_format.left_indent = indentacion
    p_fecha.add_run("Fecha:").bold = True
    
    # Obtener fecha actual en formato "día de mes de año"
    meses = ["enero", "febrero", "marzo", "abril", "mayo", "junio", 
             "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]
    hoy = datetime.datetime.now()
    fecha_formateada = f"\t {hoy.day} de {meses[hoy.month-1]} de {hoy.year}"
    
    run_fecha = p_fecha.add_run(fecha_formateada)
    run_fecha.bold = False
    #add_highlight(run_fecha, 'FFFF00')  # Yellow highlight
    # Eliminar espacio después del párrafo de fecha
    p_fecha.paragraph_format.space_after = Pt(0)
    
    # Aplicar formato Tahoma y tamaño 10 a todos los párrafos
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            run.font.name = 'Tahoma'
            run.font.size = Pt(10)

    # Añadir línea horizontal
    p_linea = doc.add_paragraph()
    p_linea.alignment = WD_ALIGN_PARAGRAPH.CENTER
    p_linea.paragraph_format.space_before = Pt(0)  # Espacio mínimo (1 punto)
    p_linea.paragraph_format.space_after = Pt(0)
    p_run = p_linea.add_run()
    
    # Configurar la línea horizontal
    p_border = OxmlElement('w:pBdr')
    p_bottom = OxmlElement('w:bottom')
    p_bottom.set(qn('w:val'), 'single')
    p_bottom.set(qn('w:sz'), '4')
    p_bottom.set(qn('w:space'), '1')
    p_bottom.set(qn('w:color'), 'auto')
    p_border.append(p_bottom)
    
    p_pr = p_linea._element.get_or_add_pPr()
    p_pr.append(p_border)
    
    # Resto del documento (igual que antes)...
    # Párrafo de saludo
    p_saludo = doc.add_paragraph("De mi mayor consideración:")
    p_saludo.runs[0].font.name = 'Tahoma'
    p_saludo.runs[0].font.size = Pt(10)
    
    # Párrafo principal
    p_principal = doc.add_paragraph()
    p_principal.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_principal = p_principal.add_run(
        "En cumplimiento a lo establecido en el Reglamento para la aplicación técnica y la gestión "
        "administrativa y financiera de la Ley N° 1152, aprobado mediante Resolución Ministerial N° "
        "0251 de 30 de junio de 2021, remito el informe de revisión del cumplimiento de la Correlación "
        "Clínica Administrativa de los Servicios de Salud otorgados a la población beneficiaria, en el "
        "Hospital Municipal de Segundo Nivel La Portada la gestión 2025:"
    )
    run_principal.font.name = 'Tahoma'
    run_principal.font.size = Pt(10)
    
    # Sección 1. ANTECEDENTES
    p_ant_titulo = doc.add_paragraph("      1. ANTECEDENTES")
    p_ant_titulo.runs[0].font.name = 'Tahoma'
    p_ant_titulo.runs[0].font.size = Pt(10)
    p_ant_titulo.runs[0].font.bold = True
    
    # Párrafo 1 de antecedentes
    p_ant1 = doc.add_paragraph()
    p_ant1.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_ant1 = p_ant1.add_run(
        "La Ley N° 475, de 30 de diciembre de 2013, de Prestaciones de Servicios de Salud Integral del "
        "Estado Plurinacional de Bolivia, tiene por objeto establecer y regular la atención integral y la "
        "protección financiera en salud de la población beneficiaria que no se encuentra cubierta por el "
        "Seguro Social Obligatorio de Corto Plazo y establecer las bases para la universalización de la "
        "atención integral en salud."
    )
    run_ant1.font.name = 'Tahoma'
    run_ant1.font.size = Pt(10)
    
    # Párrafo 2 de antecedentes
    p_ant2 = doc.add_paragraph()
    p_ant2.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_ant2 = p_ant2.add_run(
        "La Ley N° 1055, de 28 de mayo de 2018, modifica la Ley N° 475, para optimizar el uso de los "
        "recursos financieros asignados a la atención integral de salud."
    )
    run_ant2.font.name = 'Tahoma'
    run_ant2.font.size = Pt(10)
    
    # Párrafo 3 de antecedentes
    p_ant3 = doc.add_paragraph()
    p_ant3.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_ant3 = p_ant3.add_run(
        "La Ley N° 1152, de 20 de febrero de 2019, modifica la Ley N° 475, para ampliar la población "
        "beneficiaria que no se encuentra cubierta por la Seguridad Social de Corto Plazo, con atención "
        "gratuita en salud, en avance hacia un Sistema Único de Salud, Universal y Gratuito."
    )
    run_ant3.font.name = 'Tahoma'
    run_ant3.font.size = Pt(10)
    
    # Párrafo 4 de antecedentes
    p_ant4 = doc.add_paragraph()
    p_ant4.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_ant4 = p_ant4.add_run(
        "La Resolución Ministerial N° 0251, de 30 de junio de 2021, aprueba el Reglamento para la "
        "Aplicación Técnica y la Gestión Administrativa y Financiera de la Ley N° 1152 de 20 de febrero "
        "de 2019 \"hacia un Sistema Único de Salud Universal y Gratuito."
    )
    run_ant4.font.name = 'Tahoma'
    run_ant4.font.size = Pt(10)
    
    # Sección 2. JUSTIFICACIÓN
    p_just_titulo = doc.add_paragraph("       2. JUSTIFICACIÓN")
    p_just_titulo.runs[0].font.name = 'Tahoma'
    p_just_titulo.runs[0].font.size = Pt(10)
    p_just_titulo.runs[0].font.bold = True
    
    # Párrafo de justificación
    p_just = doc.add_paragraph()
    p_just.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_just = p_just.add_run(
        "La correlación clínica administrativa, es la coherencia existente entre los diagnósticos y "
        "procedimientos efectuados por el personal de salud, registrados en el expediente clínico, que "
        "comprende la historia clínica, epicrisis, hojas de enfermería, hojas de evolución y otros "
        "pertinentes a la atención, con los servicios de salud declarados en los documentos "
        "administrativos generados por las herramientas informáticas del establecimiento; la misma que "
        "debe ser realizada el propósito de mejorar la calidad del dato y la atención de los pacientes "
        "y retroalimentar al personal de salud y administrativo."
    )
    run_just.font.name = 'Tahoma'
    run_just.font.size = Pt(10)
    
    # PÁGINA 2
    doc.add_paragraph()
    doc.add_paragraph()
    # Párrafo inicial sobre el Hospital
    p1 = doc.add_paragraph()
    p1.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run1 = p1.add_run("El Hospital La Portada, en cumplimiento al Parágrafo VI, inciso b) del Artículo 53 del Reglamento para la Aplicación Técnica y la Gestión Administrativa y Financiera de la Ley N° 1152, presenta a las entidades técnicas para el proceso de cobro interinstitucional a las GAIs o GAICs deudores, el informe de Correlación Clínica Administrativa de los pacientes referidos y en tránsito de dichos municipios.")
    run1.font.name = 'Tahoma'
    run1.font.size = Pt(10)

    # 3. METODOLOGÍA
    p_met = doc.add_paragraph()
    run_met = p_met.add_run("       3. METODOLOGIA")
    run_met.bold = True
    run_met.font.name = 'Tahoma'
    run_met.font.size = Pt(10)

    # Párrafo metodología
    p_met_desc = doc.add_paragraph()
    p_met_desc.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_met_desc = p_met_desc.add_run("El Informe de Correlación Clínica Administrativa, se elaboró bajo la siguiente metodología:")
    run_met_desc.font.name = 'Tahoma'
    run_met_desc.font.size = Pt(10)

    # Lista a)
    p_a = doc.add_paragraph(style='List')
    p_a.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    p_a.paragraph_format.left_indent = Inches(0.25)
    p_a.paragraph_format.first_line_indent = Inches(-0.25)
    run_a = p_a.add_run("a)  Se realizó la revisión de los expedientes clínicos de pacientes atendidos según incisos a) y b) del parágrafo 1 del artículo 53 del Reglamento para la Aplicación Técnica y la Gestión Administrativa y Financiera de la Ley N° 1152, incluidos en los cobros intermunicipales y se los comparó con los registros del SIAF del establecimiento.")
    run_a.font.name = 'Tahoma'
    run_a.font.size = Pt(10)

    # Lista b)
    p_b = doc.add_paragraph(style='List')
    p_b.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    p_b.paragraph_format.left_indent = Inches(0.25)
    p_b.paragraph_format.first_line_indent = Inches(-0.25)
    run_b = p_b.add_run("b)  Para este fin se aplicó al instrumento de verificación proporcionado por el Ministerio de Salud y Deportes, que incluye las siguientes variables:")
    run_b.font.name = 'Tahoma'
    run_b.font.size = Pt(10)

    # Viñetas bajo b) - con sangría adicional
    bullets = [
        "Verificación de los datos generales de cada paciente.",
        "Identificación del Diagnóstico Principal.",
        "Identificación de posibles complicaciones y comorbilidades del paciente.",
        "Procedimientos terapéuticos y diagnósticos otorgados."
    ]

    for bullet in bullets:
        p_bullet = doc.add_paragraph(style='List Bullet 2')  # Nivel más profundo de viñeta
        p_bullet.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        p_bullet.paragraph_format.left_indent = Inches(0.5)
        p_bullet.paragraph_format.first_line_indent = Inches(-0.25)
        run_bullet = p_bullet.add_run(bullet)
        run_bullet.font.name = 'Tahoma'
        run_bullet.font.size = Pt(10)

    # Lista c) y d)
    items_cd = [
        ("c)   Identificación y descripción de las principales inconsistencias encontradas."),
        ("d)   Descripción de las acciones correctivas realizadas en el establecimiento de salud.")
    ]

    for item in items_cd:
        p_item = doc.add_paragraph(style='List')
        p_item.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        p_item.paragraph_format.left_indent = Inches(0.25)
        p_item.paragraph_format.first_line_indent = Inches(-0.25)
        run_item = p_item.add_run(item)
        run_item.font.name = 'Tahoma'
        run_item.font.size = Pt(10)

    # 4. ANÁLISIS TÉCNICO
    p_analisis = doc.add_paragraph()
    run_analisis = p_analisis.add_run("     4. ANÁLISIS TÉCNICO")
    run_analisis.bold = True
    run_analisis.font.name = 'Tahoma'
    run_analisis.font.size = Pt(10)

    # RESULTADOS
    p_resultados = doc.add_paragraph()
    run_resultados = p_resultados.add_run("RESULTADOS")
    run_resultados.bold = True
    run_resultados.font.name = 'Tahoma'
    run_resultados.font.size = Pt(10)

    # Párrafo de resultados
    p_res_desc = doc.add_paragraph()
    p_res_desc.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_res_desc = p_res_desc.add_run("De la revisión de la muestra señalada de los expedientes clínicos y de los servicios de salud declarados en el SIAF del establecimiento de salud, se obtuvieron los siguientes resultados generales:")
    run_res_desc.font.name = 'Tahoma'
    run_res_desc.font.size = Pt(10)

    # Acciones correctivas realizadas
    p_acciones = doc.add_paragraph()
    run_acciones = p_acciones.add_run("Acciones correctivas realizadas")
    run_acciones.bold = True
    run_acciones.font.name = 'Tahoma'
    run_acciones.font.size = Pt(10)

    # Párrafo acciones
    p_acc_desc = doc.add_paragraph()
    p_acc_desc.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_acc_desc = p_acc_desc.add_run("Con los resultados de la revisión de la correlación clínica administrativa del hospital de Segundo Nivel La Portada, se realizaron las siguientes acciones correctivas por parte de la Dirección del establecimiento de salud, con insumos proporcionados por el responsable del SUS del Hospital:")
    run_acc_desc.font.name = 'Tahoma'
    run_acc_desc.font.size = Pt(10)

    # Punto 1
    p_punto1 = doc.add_paragraph()
    p_punto1.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_punto1_num = p_punto1.add_run("\t1. ")
    run_punto1_num.font.name = 'Tahoma'
    run_punto1_num.font.size = Pt(10)
    run_punto1_text = p_punto1.add_run(" Se retirará del cobro intermunicipal al GAM o GAIOC deudor, aquellos expedientes \t\t     clínicos que presentan observaciones.")
    run_punto1_text.font.name = 'Tahoma'
    run_punto1_text.font.size = Pt(10)

    # CONCLUSIONES
    p_conclusiones = doc.add_paragraph()
    run_conclusiones = p_conclusiones.add_run("CONCLUSIONES")
    run_conclusiones.bold = True
    run_conclusiones.font.name = 'Tahoma'
    run_conclusiones.font.size = Pt(10)

    # Conclusión 1 - blaTem1
    p_conc1 = doc.add_paragraph(style='List Number')
    p_conc1.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_conc1_num = p_conc1.add_run("Se realizó la verificación de la posible resistencia de la bacteria ")
    run_conc1_num.font.name = 'Tahoma'
    run_conc1_num.font.size = Pt(10)

    run_conc1_highlight = p_conc1.add_run(nombre_cepa)  # Nombre de la cepa aquí
    run_conc1_highlight.font.name = 'Tahoma'
    run_conc1_highlight.font.size = Pt(10)
    #add_highlight(run_conc1_highlight, 'FFFF00')  # Amarillo

    run_conc1_rest = p_conc1.add_run(". De la que se sospecha posee el gen de resistencia \"blaTem1\", del análisis genético realizado se puede concluir que esta es:")
    run_conc1_rest.font.name = 'Tahoma'
    run_conc1_rest.font.size = Pt(10)

    # POSITIVO/NEGATIVO 1 - blaTem1
    p_pos_neg1 = doc.add_paragraph()
    p_pos_neg1.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Determinar el color basado en el resultado
    if resultado_blatem1.upper() == "SI" or resultado_blatem1.upper() == "SÍ":
        texto_resultado1 = "POSITIVO"
        color_highlight1 = 'FF0000'  # Rojo para positivo
    else:
        texto_resultado1 = "NEGATIVO"
        color_highlight1 = '00FF00'  # Verde para negativo
    
    run_pos_neg1 = p_pos_neg1.add_run(texto_resultado1)
    run_pos_neg1.font.name = 'Tahoma'
    run_pos_neg1.font.size = Pt(10)
    run_pos_neg1.bold = True
    add_highlight(run_pos_neg1, color_highlight1)

    # Conclusión 2 - blaCTXM15
    p_conc2 = doc.add_paragraph(style='List Number')
    p_conc2.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
    run_conc2_num = p_conc2.add_run("Se realizó la verificación de la posible resistencia de la bacteria ")
    run_conc2_num.font.name = 'Tahoma'
    run_conc2_num.font.size = Pt(10)

    run_conc2_highlight = p_conc2.add_run(nombre_cepa)  # Nombre de la cepa aquí
    run_conc2_highlight.font.name = 'Tahoma'
    run_conc2_highlight.font.size = Pt(10)
    #add_highlight(run_conc2_highlight, 'FFFF00')  # Amarillo

    run_conc2_rest = p_conc2.add_run(". De la que se sospecha posea el gen de resistencia \"blaCTXM15\", del análisis genético realizado se puede concluir que esta es:")
    run_conc2_rest.font.name = 'Tahoma'
    run_conc2_rest.font.size = Pt(10)

    # POSITIVO/NEGATIVO 2 - blaCTXM15
    p_pos_neg2 = doc.add_paragraph()
    p_pos_neg2.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # Determinar el color basado en el resultado
    if resultado_blactxm15.upper() == "SI" or resultado_blactxm15.upper() == "SÍ":
        texto_resultado2 = "POSITIVO"
        color_highlight2 = 'FF0000'  # Rojo para positivo
    else:
        texto_resultado2 = "NEGATIVO"
        color_highlight2 = '00FF00'  # Verde para negativo
    
    run_pos_neg2 = p_pos_neg2.add_run(texto_resultado2)
    run_pos_neg2.font.name = 'Tahoma'
    run_pos_neg2.font.size = Pt(10)
    run_pos_neg2.bold = True
    add_highlight(run_pos_neg2, color_highlight2)
    
    doc.add_paragraph()
    doc.add_paragraph()

    # Línea con texto azul
    p_ingeniero = doc.add_paragraph()
    p_ingeniero.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run_ingeniero = p_ingeniero.add_run("Valeria Avril Candia Zarco")
    run_ingeniero.font.name = 'Tahoma'
    run_ingeniero.font.size = Pt(10)
    #run_ingeniero.font.color.rgb = RGBColor(0, 0, 255)  # Azul
    #run_ingeniero.underline = True

    # C.I.
    p_ci = doc.add_paragraph()
    p_ci.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run_ci = p_ci.add_run("C.I.: 6899414 L.P")
    run_ci.font.name = 'Tahoma'
    run_ci.font.size = Pt(10)

    # Archivo
    p_archivo = doc.add_paragraph()
    p_archivo.alignment = WD_ALIGN_PARAGRAPH.LEFT
    run_archivo_cc = p_archivo.add_run("Cc: ")
    run_archivo_cc.font.name = 'Arial Narrow'
    run_archivo_cc.font.size = Pt(8)
    run_archivo = p_archivo.add_run("Archivo")
    run_archivo.font.name = 'Arial Narrow'
    run_archivo.font.size = Pt(8)

    return doc

def generar_informes_por_cepa(archivo_csv="resultados_resistencia.csv"):
    """Función principal para generar los informes por cada cepa"""
    # Leer los resultados del CSV
    resultados = leer_resultados_csv(archivo_csv)
    
    if not resultados:
        print("No se pudieron leer los datos del CSV. Verificar el archivo.")
        return
    
    # Crear directorio para los informes si no existe
    if not os.path.exists("informes_cepas"):
        os.makedirs("informes_cepas")
    
    documentos_generados = []
    
    # Generar un documento para cada cepa
    for i, resultado in enumerate(resultados, 1):
        nombre_cepa = resultado['nombre_cepa']
        resultado_blatem1 = resultado['blatem1']
        resultado_blactxm15 = resultado['blactxm15']
        
        print(f"Generando informe {i}/8 para la cepa: {nombre_cepa}")
        
        # Crear el documento
        documento = crear_informe_correlacion(nombre_cepa, resultado_blatem1, resultado_blactxm15)
        
        # Nombre del archivo
        nombre_archivo = f"informes_cepas/Informe_bacteria_{nombre_cepa}.docx"
        
        # Guardar el documento
        documento.save(nombre_archivo)
        documentos_generados.append(nombre_archivo)
        
        print(f"  ✓ Documento generado: {nombre_archivo}")
        print(f"  - blaTem1: {resultado_blatem1}")
        print(f"  - blaCTXM15: {resultado_blactxm15}")
        print()
    
    print("="*50)
    print("RESUMEN DE GENERACIÓN DE INFORMES")
    print("="*50)
    print(f"Total de documentos generados: {len(documentos_generados)}")
    print("\nArchivos creados:")
    for doc in documentos_generados:
        print(f"  - {doc}")
    
    print(f"\nTodos los documentos se han guardado en la carpeta 'informes_cepas/'")
    return documentos_generados

# Función para verificar el archivo CSV antes de procesar
def verificar_csv(archivo_csv="resultados_resistencia.csv"):
    """Verifica que el archivo CSV exista y tenga el formato correcto"""
    try:
        with open(archivo_csv, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            filas = list(reader)
            
            print("VERIFICACIÓN DEL ARCHIVO CSV")
            print("="*40)
            print(f"Archivo: {archivo_csv}")
            print(f"Número de cepas encontradas: {len(filas)}")
            print()
            
            if len(filas) == 0:
                print("❌ ERROR: El archivo CSV está vacío")
                return False
            
            # Verificar columnas necesarias
            columnas_requeridas = ['Genoma', 'blatem1', 'blactxm15']
            columnas_encontradas = reader.fieldnames
            
            print("Columnas encontradas:", columnas_encontradas)
            print("Columnas requeridas:", columnas_requeridas)
            
            for col in columnas_requeridas:
                if col not in columnas_encontradas:
                    print(f"❌ ERROR: Falta la columna '{col}' en el archivo CSV")
                    return False
            
            print("✓ Formato del CSV correcto")
            print()
            
            # Mostrar preview de los datos
            print("PREVIEW DE LOS DATOS:")
            print("-" * 80)
            for i, fila in enumerate(filas[:5], 1):  # Mostrar solo las primeras 5
                nombre_completo = fila['Genoma']
                if '_' in nombre_completo:
                    partes = nombre_completo.split('_')
                    nombre_cepa = partes[1] if len(partes) >= 2 else partes[0]
                else:
                    nombre_cepa = nombre_completo.replace('.fna', '')
                
                print(f"{i}. Cepa: {nombre_cepa}")
                print(f"   Genoma: {nombre_completo}")
                print(f"   blaTem1: {fila['blatem1']}")
                print(f"   blaCTXM15: {fila['blactxm15']}")
                print()
            
            if len(filas) > 5:
                print(f"... y {len(filas) - 5} cepas más")
            
            return True
            
    except FileNotFoundError:
        print(f"❌ ERROR: No se encontró el archivo '{archivo_csv}'")
        print("Asegúrese de que el archivo esté en la misma carpeta que este script.")
        return False
    except Exception as e:
        print(f"❌ ERROR al leer el archivo CSV: {e}")
        return False

# Función principal ejecutable
def main():
    """Función principal que ejecuta todo el proceso"""
    archivo_csv = "resultados_resistencia.csv"
    
    print("GENERADOR DE INFORMES DE RESISTENCIA BACTERIANA")
    print("=" * 50)
    print()
    
    # Verificar el archivo CSV primero
    if not verificar_csv(archivo_csv):
        print("❌ No se puede continuar debido a problemas con el archivo CSV.")
        return
    
    # Generar los informes
    print("Iniciando generación de informes...")
    print()
    
    try:
        documentos = generar_informes_por_cepa(archivo_csv)
        
        if documentos:
            print("✅ PROCESO COMPLETADO EXITOSAMENTE")
            print()
            print("Los informes están listos para su revisión.")
        else:
            print("❌ No se generaron documentos.")
            
    except Exception as e:
        print(f"❌ ERROR durante la generación: {e}")
        print("Verifique que:")
        print("  - El archivo CSV tenga el formato correcto")
        print("  - Tenga permisos de escritura en la carpeta")
        print("  - La librería python-docx esté instalada")

# Ejecutar el programa
if __name__ == "__main__":
    main()

GENERADOR DE INFORMES DE RESISTENCIA BACTERIANA

VERIFICACIÓN DEL ARCHIVO CSV
Archivo: resultados_resistencia.csv
Número de cepas encontradas: 8

Columnas encontradas: ['Genoma', 'blatem1', 'blactxm15']
Columnas requeridas: ['Genoma', 'blatem1', 'blactxm15']
✓ Formato del CSV correcto

PREVIEW DE LOS DATOS:
--------------------------------------------------------------------------------
1. Cepa: 050389795.1
   Genoma: GCF_050389795.1_MS10495_ARL02201_ST410_genomic.fna
   blaTem1: No
   blaCTXM15: No

2. Cepa: 050516345.1
   Genoma: GCF_050516345.1_ASM5051634v1_genomic.fna
   blaTem1: Sí
   blaCTXM15: No

3. Cepa: 050516355.1
   Genoma: GCF_050516355.1_ASM5051635v1_genomic.fna
   blaTem1: No
   blaCTXM15: No

4. Cepa: 050516365.1
   Genoma: GCF_050516365.1_ASM5051636v1_genomic.fna
   blaTem1: Sí
   blaCTXM15: No

5. Cepa: 050516375.1
   Genoma: GCF_050516375.1_ASM5051637v1_genomic.fna
   blaTem1: No
   blaCTXM15: No

... y 3 cepas más
Iniciando generación de informes...

Generando infor